In [ ]:
from google.cloud import storage
from dotenv import load_dotenv
import os
from pathlib import Path
import uuid
from Multi_Modal.chunking import get_chunks

In [ ]:
load_dotenv()

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
PDF_BUCKET_NAME = os.environ['PDF_BUCKET_NAME']
project_id=os.environ['PROJECT_ID']

In [ ]:
from pathlib import Path
from google.cloud import storage
import uuid

def upload_file_to_gcp(local_file_path):
    try:
        unique_value = str(uuid.uuid4())
        
        file_name = Path(local_file_path).name
        dest_name = f"uploads/{unique_value}_{file_name}"
        
        storage_client = storage.Client(project=project_id)
        bucket = storage_client.bucket(PDF_BUCKET_NAME)

        print("Successfully logged in gcp")
        print(dest_name)

        blob = bucket.blob(dest_name)

        blob.upload_from_filename(str(local_file_path))

        print(f"File {local_file_path} uploaded to gs://{PDF_BUCKET_NAME}/{dest_name}.")
        return f"gs://{PDF_BUCKET_NAME}/{dest_name}"

    except Exception as e:
        print("Could not upload file to GCP:", e)


In [ ]:
path="D:/Cite-What-You-Type/pdfs/15. K Means clustering Algorithm.pptx"

In [ ]:
uploded_path =upload_file_to_gcp(path)

In [ ]:
uploded_path

In [ ]:
# get_chunks(uploded_path)